In [5]:
import os

os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= '1' # 


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer
torch.zeros(1).cuda()

torch.cuda.empty_cache()

base_model_id = "mistralai/Mixtral-8x7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="cuda")

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 79.21 GiB total capacity; 23.22 GiB already allocated; 15.62 MiB free; 23.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [3]:
# Re-init the tokenizer so it doesn't add padding or eos token
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

In [4]:
torch.zeros(1).cuda()
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='training.json', split='train')

In [5]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mixtral-RadMix/checkpoint-300")

NameError: name 'base_model' is not defined

In [6]:
from datasets import load_dataset
eval_dataset = load_dataset('json', data_files='testing.json', split='train')
eval_dataset

Dataset({
    features: ['Findings', 'Medical Condition'],
    num_rows: 3612
})

In [14]:
eval_prompt = "48 year old man with HCV s/p liver transplant in who now has severe headaches REASON FOR THIS EXAMINATION: Liver transplant patient now with severe headaches BRAIN VOLUMETRIC DATA: Grey Matter: 762.36 White matter: 582.97 csf: 0.71 Total Brain Volume: 1345.33 Total Intracranial Volume: 1910.4 Total Ventricular Volume: 13.22 Lateral Ventricles Volume: 10.45 First Ventricle Volume: 3.77 Second Ventricle Volume: 6.68 Third Ventricle Volume: 0.92 Fourth Ventricle Volume: 1.857 /n ###Generate a findings section of the radiological report and add in volumetric data where applicable."
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=150, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


48 year old man with HCV s/p liver transplant in who now has severe headaches REASON FOR THIS EXAMINATION: Liver transplant patient now with severe headaches BRAIN VOLUMETRIC DATA: Grey Matter: 762.36 White matter: 582.97 csf: 0.71 Total Brain Volume: 1345.33 Total Intracranial Volume: 1910.4 Total Ventricular Volume: 13.22 Lateral Ventricles Volume: 10.45 First Ventricle Volume: 3.77 Second Ventricle Volume: 6.68 Third Ventricle Volume: 0.92 Fourth Ventricle Volume: 1.857 /n ###Generate a findings section of the radiological report and add in volumetric data where applicable. ### Sample Output: There is no evidence for acute intracranial hemorrhage, mass effect or shift of normally midline structures. The ventricles are normal in size and configuration. No major vascular territorial infarcts are identified. A small focus of hypodensity within the left frontoparietal lobe may represent an area of encephalomalacia from prior trauma (see previous study). Small calcifications along the te

In [17]:
import json
generated_texts = {}
for idx, example in enumerate(eval_dataset):
    eval_prompt = f"Generate a findings section based on the medical condition: {example['Medical Condition']} and add in volumetric data where applicable. ### Sample output:"
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    model.eval()
    with torch.no_grad():
        generated_texts[f"{idx}"] = tokenizer.decode(model.generate(**model_input, max_new_tokens=250, repetition_penalty=1.15)[0], skip_special_tokens=True)
    print(f"Completed generation for example {idx}")
    if idx == 149:  
        break

with open("generated_texts2.json", "w") as f:
    json.dump(generated_texts, f)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


FP4 quantization state not initialized. Please call .cuda() or .to(device) on the LinearFP4 layer first.


AssertionError: 

In [7]:
eval_prompt = "48 year old man with HCV s/p liver transplant in who now has severe headaches REASON FOR THIS EXAMINATION: Liver transplant patient now with severe headaches BRAIN VOLUMETRIC DATA: Grey Matter: 762.36 White matter: 582.97 csf: 0.71 Total Brain Volume: 1345.33 Total Intracranial Volume: 1910.4 Total Ventricular Volume: 13.22 Lateral Ventricles Volume: 10.45 First Ventricle Volume: 3.77 Second Ventricle Volume: 6.68 Third Ventricle Volume: 0.92 Fourth Ventricle Volume: 1.857 /n ###Generate a findings section of the radiological report and add in volumetric data where applicable."
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=150, repetition_penalty=1.15)[0], skip_special_tokens=True))

NameError: name 'model' is not defined

In [13]:
import json
import re
from nltk.translate.bleu_score import corpus_bleu

with open("generated_texts.json", "r") as f:
    generated_texts = json.load(f)


generated_summaries = []
for text in generated_texts.values():
    match = re.search(r'#+\s*(.*)', text)
    if match:
        generated_summaries.append(match.group(1).strip())
    else:
        generated_summaries.append(text) 
    


generated_summaries = [s.replace("Sample Output: ", "") for s in generated_summaries]

print("Generated Summaries:", generated_summaries)




Generated Summaries: ['There are no intraparenchymal hemorrhages, mass lesions or acute major vascular territorial infarcts identified on this study. The ventricles and sulci appear normal in size and configuration with mild periventricular white matter (446.24) hypodensities consistent with chronic microvascular disease. A small focus of hyperdensity within the left frontoparietal subcortical region likely represents an area of prior infarction. Small calcifications along the tentorium cerebelli bilaterally may represent dystrophic changes from previous bleeds.  Bone windows demonstrate diffuse osteopenia without evidence of fracture. Soft tissue structures reveal soft tissue swelling overlying the right parieto-occipital scalp which could be secondary to trauma but more likely due to postoperative change given recent craniotomy.  Posterior fossa cisterns remain patent. Visualized portions of the mastoid air cells show mucosal thickening compatible with sinusitis', 'There is no eviden

In [10]:
with open("testing.json", "r") as f:
    testing_data = json.load(f)

reference_summaries = [example['Findings'] for example in testing_data[:150]]

reference_summaries

['No acute intracranial hemorrhage, large vascular territory infarct, edema or mass effect is present.  The ventricles and sulci are normal in size and configuration.  Chronic lacunar infarcts are noted in the right putamen and left subinsular white matter (446.24).  There are mucus retention cysts in both maxillary sinuses as well as air-fluid levels in the frontal, ethmoidal and sphenoid sinuses, likely related to endotracheal intubation.',
 'There is a hyperdensity along the convexity of the right frontal, parietal, and temporal lobes with some areas of hypodensity that is consistent with an acute on chronic subdural hematoma.  This extra-axial collection is unchanged in thickness, extent, and mass effect compared to prior study.  It extends from the vertex to the middle cranial fossa.  There is effacement of the adjacent sulci and a shift in the normally midline structures 6 mm to the left, this is unchanged from prior study. There is also a small hyperdense extra-axial collection 

In [21]:
firstGenerated = generated_summaries[0]
firstGenerated

'There are no intraparenchymal hemorrhages, mass lesions or acute major vascular territorial infarcts identified on this study. The ventricles and sulci appear normal in size and configuration with mild periventricular white matter (446.24) hypodensities consistent with chronic microvascular disease. A small focus of hyperdensity within the left frontoparietal subcortical region likely represents an area of prior infarction. Small calcifications along the tentorium cerebelli bilaterally may represent dystrophic changes from previous bleeds.  Bone windows demonstrate diffuse osteopenia without evidence of fracture. Soft tissue structures reveal soft tissue swelling overlying the right parieto-occipital scalp which could be secondary to trauma but more likely due to postoperative change given recent craniotomy.  Posterior fossa cisterns remain patent. Visualized portions of the mastoid air cells show mucosal thickening compatible with sinusitis'

In [22]:
firstReference = reference_summaries[0]
firstReference

'No acute intracranial hemorrhage, large vascular territory infarct, edema or mass effect is present.  The ventricles and sulci are normal in size and configuration.  Chronic lacunar infarcts are noted in the right putamen and left subinsular white matter (446.24).  There are mucus retention cysts in both maxillary sinuses as well as air-fluid levels in the frontal, ethmoidal and sphenoid sinuses, likely related to endotracheal intubation.'

In [23]:
reference = [firstGenerated.split()]
reference

[['There',
  'are',
  'no',
  'intraparenchymal',
  'hemorrhages,',
  'mass',
  'lesions',
  'or',
  'acute',
  'major',
  'vascular',
  'territorial',
  'infarcts',
  'identified',
  'on',
  'this',
  'study.',
  'The',
  'ventricles',
  'and',
  'sulci',
  'appear',
  'normal',
  'in',
  'size',
  'and',
  'configuration',
  'with',
  'mild',
  'periventricular',
  'white',
  'matter',
  '(446.24)',
  'hypodensities',
  'consistent',
  'with',
  'chronic',
  'microvascular',
  'disease.',
  'A',
  'small',
  'focus',
  'of',
  'hyperdensity',
  'within',
  'the',
  'left',
  'frontoparietal',
  'subcortical',
  'region',
  'likely',
  'represents',
  'an',
  'area',
  'of',
  'prior',
  'infarction.',
  'Small',
  'calcifications',
  'along',
  'the',
  'tentorium',
  'cerebelli',
  'bilaterally',
  'may',
  'represent',
  'dystrophic',
  'changes',
  'from',
  'previous',
  'bleeds.',
  'Bone',
  'windows',
  'demonstrate',
  'diffuse',
  'osteopenia',
  'without',
  'evidence',
  '

In [24]:
from nltk.translate.bleu_score import sentence_bleu

candidate = firstReference.split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

BLEU score -> 0.03930882145080463


In [25]:
generated_tokenized = [text.split() for text in generated_summaries]
generated_tokenized

[['There',
  'are',
  'no',
  'intraparenchymal',
  'hemorrhages,',
  'mass',
  'lesions',
  'or',
  'acute',
  'major',
  'vascular',
  'territorial',
  'infarcts',
  'identified',
  'on',
  'this',
  'study.',
  'The',
  'ventricles',
  'and',
  'sulci',
  'appear',
  'normal',
  'in',
  'size',
  'and',
  'configuration',
  'with',
  'mild',
  'periventricular',
  'white',
  'matter',
  '(446.24)',
  'hypodensities',
  'consistent',
  'with',
  'chronic',
  'microvascular',
  'disease.',
  'A',
  'small',
  'focus',
  'of',
  'hyperdensity',
  'within',
  'the',
  'left',
  'frontoparietal',
  'subcortical',
  'region',
  'likely',
  'represents',
  'an',
  'area',
  'of',
  'prior',
  'infarction.',
  'Small',
  'calcifications',
  'along',
  'the',
  'tentorium',
  'cerebelli',
  'bilaterally',
  'may',
  'represent',
  'dystrophic',
  'changes',
  'from',
  'previous',
  'bleeds.',
  'Bone',
  'windows',
  'demonstrate',
  'diffuse',
  'osteopenia',
  'without',
  'evidence',
  '

In [26]:
reference_tokenized = [text.split() for text in reference_summaries]
reference_tokenized

[['No',
  'acute',
  'intracranial',
  'hemorrhage,',
  'large',
  'vascular',
  'territory',
  'infarct,',
  'edema',
  'or',
  'mass',
  'effect',
  'is',
  'present.',
  'The',
  'ventricles',
  'and',
  'sulci',
  'are',
  'normal',
  'in',
  'size',
  'and',
  'configuration.',
  'Chronic',
  'lacunar',
  'infarcts',
  'are',
  'noted',
  'in',
  'the',
  'right',
  'putamen',
  'and',
  'left',
  'subinsular',
  'white',
  'matter',
  '(446.24).',
  'There',
  'are',
  'mucus',
  'retention',
  'cysts',
  'in',
  'both',
  'maxillary',
  'sinuses',
  'as',
  'well',
  'as',
  'air-fluid',
  'levels',
  'in',
  'the',
  'frontal,',
  'ethmoidal',
  'and',
  'sphenoid',
  'sinuses,',
  'likely',
  'related',
  'to',
  'endotracheal',
  'intubation.'],
 ['There',
  'is',
  'a',
  'hyperdensity',
  'along',
  'the',
  'convexity',
  'of',
  'the',
  'right',
  'frontal,',
  'parietal,',
  'and',
  'temporal',
  'lobes',
  'with',
  'some',
  'areas',
  'of',
  'hypodensity',
  'that'

In [27]:
bleu_score = corpus_bleu([[ref] for ref in reference_tokenized], generated_tokenized)
bleu_score

0.04854566587719963

In [18]:
from rouge_score import rouge_scorer

In [28]:
total_scores = {metric: 0 for metric in ['rouge1', 'rouge2', 'rougeL']}
num_pairs = len(generated_summaries)
for gen_summary, ref_summary in zip(generated_summaries, reference_summaries):

    scores = scorer.score(gen_summary, ref_summary)
    for metric in scores:
        total_scores[metric] += scores[metric].fmeasure

average_scores = {metric: total_scores[metric] / num_pairs for metric in total_scores}

for metric in average_scores:
    print(f'Average {metric} score: {average_scores[metric]}')

Average rouge1 score: 0.31777233540665534
Average rouge2 score: 0.10325957688739164
Average rougeL score: 0.18565165040624776


In [19]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [20]:
scores = scorer.score(generated_summaries[0], reference_summaries[0])
for key in scores:
    print(f'{key}: {scores[key]}')

rouge1: Score(precision=0.4925373134328358, recall=0.2578125, fmeasure=0.3384615384615385)
rouge2: Score(precision=0.21212121212121213, recall=0.11023622047244094, fmeasure=0.14507772020725387)
rougeL: Score(precision=0.31343283582089554, recall=0.1640625, fmeasure=0.2153846153846154)


In [31]:
from bert_score import score
P, R, F1 = score(generated_summaries, reference_summaries, lang='en', verbose=True)

print('Precision:', P.mean())
print('Recall:', R.mean())
print('F1 Score:', F1.mean())

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/5 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 4.26 seconds, 35.19 sentences/sec
Precision: tensor(0.8503)
Recall: tensor(0.8626)
F1 Score: tensor(0.8562)


In [ ]:
base_model_id = "mistralai/Mixtral-8x7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="cuda")

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
import json
generated_texts = {}
for idx, example in enumerate(eval_dataset):
    model_input = tokenizer(example['Medical Condition'], return_tensors="pt").to("cuda")
    model.eval()
    with torch.no_grad():
        generated_texts[f"{idx}"] = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=250, repetition_penalty=1.15)[0], skip_special_tokens=True)
    print(f"Completed generation for example {idx}")
    if idx == 99:  
        break

with open("generated_texts2.json", "w") as f:
    json.dump(generated_texts, f)